# Домашнее задание №3
## Знакомство с языком обработки данных SAS Base
### Набегающие суммы. Агрегирование данных. Сортировка. Циклы. Массивы.

_[Гончаренко Дмитрий Александрович, Жуков Павел Николаевич, Слабоузова Анна Константиновна]_

### 0. Подготовка файлов для выполнения ДЗ.
Вместе с данным файлом в архиве находятся данные для выполнения ДЗ: 
- 4 набора данных SAS: **orion.orders_midyear**, **orion.order_qtrsum**, **orion.order_fact**, **orion.test_answers**;

**_Перед выполнением каждого задания познакомьтесь со структурой соответствующего входного набора - это должно ускорить выполнение! _**

Скопируйте данные в директорию на вашем компьютере, которая связана с виртуальной машиной. Далее всю работу мы будем проводить в этой директории. 

C помощью оператора `libname` создайте библиотеку SAS с именем *_Orion_*, которая "смотрит"/"является ссылкой" на директорию, в которую вы скопировали данные для выполнения задания.


In [2]:
libname Orion "/folders/myfolders/Homework_in_git/SASHW/Task3"

### 1.  Использование массивов для повторяющихся вычислений 
В наборе данных **`orion.orders_midyear`** хранится информация о сумме покупок для каждого клиента, сгруппированная по месяцам первой половины года. Менеджер по продажам Orion Star хочет на основании этих данных оценить, сколько бы потратили клиенты с учётом **5% скидки** на все товары. Требуется создать таблицу с прогнозами трат для каждого клиента в разбивке по месяцам. 

##### a.   Создайте набор данных **`discount_sales`**, учитывающий 5% скидку. 
- Создайте массив `Mon` для доступа к переменным с `Month1` по `Month6`. 
- Используйте цикл `DO` для пересчёта старых значений сумм покупок в новые. (Служебная переменная, используемая в цикле, не должна оказаться в результирующем наборе.)
- Заодно, расчитайте ожидаемую сумму покупок за все 6 месяцев для каждого клиента.

##### b.   Напечатайте первые 5 наблюдений из полученного набора SAS. 
- Добавьте подходящий заголоков к результату (оператор `title "...";`);
- Сумму покупок за 6 месяцев выведите в формате `DOLLAR12.2`;  

In [3]:
data work.discount_sales;
    set orion.orders_midyear;
    title "Custom purchases when 5% sales";
    array Mon {*} Month1 - Month6;
    format new_sum DOLLAR12.2;
    do i = 1 to DIM(Mon);
        if Mon{i} then do;
            Mon{i} = Mon{i} * 0.95;
        end;    
    end;
    new_sum = sum(of Month1 - Month6);
    drop i;
run;    

proc print data=work.discount_sales (obs=5);
run;   

Obs,Customer_ID,Month1,Month2,Month3,Month4,Month5,Month6,new_sum
1,5,202.445,.,454.10,499.51,374.63,182.200,"$1,712.89"
2,10,178.695,393.386,2733.06,3006.36,2254.77,160.825,"$8,727.09"
3,11,74.290,.,.,.,.,66.861,$141.15
4,12,128.820,.,111.72,122.89,92.17,115.938,$571.54
5,18,.,.,27.93,30.72,23.04,.,$81.70


### 2.  Агрегирование и группировка данных на шаге DATA 
Набор данных **`orion.order_qtrsum`** содержит информацию о продажах за определённый год для каждого клиента. Информация разбита по месяцам.
- В какие-то месяцы (и кварталы) клиент мог не разместить ни одного заказа.  
- Переменная `Order_Qtr` содержит соответствующий квартал.

##### a.   Создайте набор данных **`work.qtrcustomers`**, в котором подсчитывается сумма стоимости заказов каждого клиента за квартал.   
- Переменная `Total_Sales` должна содержать сумму стоимости заказов за каждый квартал для каждого значения `Customer_ID`.   
- Создайте переменную `Num_Months`, содержащую число месяцев, за которые клиент размещал хотя бы один заказ, для каждого квартала. 
- Распечатайте первые 5 наблюдений из полученного набора SAS.

In [4]:
proc sort data=orion.order_qtrsum out=work.sorted_order_qtrsum; 
    by Customer_ID;
run;       

data work.qtrcustomers;
    set work.sorted_order_qtrsum;
    title "Quarterly statistics";

    by Customer_ID;
    if first.Customer_ID then do;
        array _total_sales{*} Total_Sales1 - Total_Sales4 (4*0);
        array _num_months{*}  Num_Months1 - Num_Months4   (4*0);
    end;
    
    _qrt_i = mod(Order_Qtr, 4) + 1; 
    _total_sales{_qrt_i} = _total_sales{_qrt_i} + Sale_Amt;
    _num_months{_qrt_i} = _num_months{_qrt_i} + 1;

    if last.Customer_ID then do;
        output;
        /*как без цикла обнулить значения массива?*/
        do i = 1 to 4;
            _total_sales{i} = 0; 
            _num_months{i} =  0;
        end;    
    end;
    
    keep Customer_ID
         Total_Sales1-Total_Sales4 
         Num_Months1-Num_Months4;
run;

proc print data=work.qtrcustomers (obs=5);
run;   

Obs,Customer_ID,Total_Sales1,Total_Sales2,Total_Sales3,Total_Sales4,Num_Months1,Num_Months2,Num_Months3,Num_Months4
1,5,33.8,0.0,604.8,52.50,1,0,2,1
2,10,2037.9,32.6,342.8,1065.79,2,1,3,2
3,11,0.0,0.0,0.0,78.20,0,0,0,1
4,12,0.0,117.6,48.4,87.20,0,1,1,1
5,18,0.0,29.4,0.0,0.00,0,1,0,0


### 3.   Создание набегающих сумм по месяцам 
Набор данных **`orion.order_fact`** содержит группу заказов за несколько лет, отсортированных по `Order_Date`. 
Требуется создать набор данных, содержащий все заказы, сделанные в Orion Star за 2011 год, и набегающий итог для них: 
- Этот итог должен _сбрасываться в ноль_ в начале каждого месяца.  
- Обратите внимание, что нужно обработать только строки, в которых `Order_Date` относится к 2011 году. 
- Распечатайте первые 5 наблюдений из полученного набора SAS.

In [5]:
data work.cumsum_2011 (drop = months_passed);
    set orion.order_fact;
    title "Accumulative amount for 2011";

    format Month_Cumsum DOLLAR13.2;
    if year(Order_Date) ^= 2011 then DELETE;

    months_passed = intck('month', Lag(Order_Date), Order_Date);
    if months_passed > 0 then
        Month_Cumsum = 0;
    Month_Cumsum + Total_Retail_Price; /*какие столбцы здесь использовать чтобы посчитать сумму заказа?*/
    output;    
run;

proc print data=work.cumsum_2011 (obs=5); 
run;

Obs,Customer_ID,Employee_ID,Street_ID,Order_Date,Delivery_Date,Order_ID,Order_Type,Product_ID,Quantity,Total_Retail_Price,CostPrice_Per_Unit,Discount,Month_Cumsum
1,24,99999999,9260115784,02JAN2011,05JAN2011,1241054779,3,240800200021,2,$195.60,$42.45,.,$195.60
2,89,121135,9260116551,03JAN2011,04JAN2011,1241063739,1,240800200035,6,$160.80,$12.15,.,$356.40
3,171,120134,1600101555,04JAN2011,04JAN2011,1241066216,1,240200100225,2,$306.20,$77.85,.,$662.60
4,53,99999999,1600103258,06JAN2011,09JAN2011,1241086052,3,210200500002,3,$37.80,$5.70,.,$700.40
5,53,120131,1600103258,13JAN2011,13JAN2011,1241147641,1,240200200091,2,$362.60,$72.60,.,"$1,063.00"


### 4. Использование текстового массива для табличного поиска
Отделу общественной безопасности нужно, чтобы все сотрудники ознакомились с новыми правилами и процедурами в отношении инцидентов с клиентами в розничных магазинах. 
- Каждый сотрудник должен пройти веб-тренинг, а затем сдать тест, состоящий из 10 вопросов.  
- Каждый вопрос имеет пять вариантов ответа (от A до E).  
- Результаты тестирования находятся в наборе данных SAS **`orion.test_answers`**.  
- Каждое наблюдение в **`orion.test_answers`** содержит ответы одного человека (переменные `Q1` - `Q10`). 
- Правильные ответы на вопросы теста приведены ниже:


    Вопрос: 1 2 3 4 5 6 7 8 9 10 
    Ответ:  A C C B E E D B B A 
    
##### a. Определите, кто из сотрудников прошёл или не прошёл этот тест.  
- Вычислите число баллов за тест как число правильных ответов для каждого человека. 
    _**Замечание:** Создайте временный массив для хранения правильных ответов к тесту._  
- Если число баллов, полученных сотрудником, 7 или выше, запишите это наблюдение в набор данных **`passed`**. 
- Если сотрудник набрал менее 7 баллов, запишите это наблюдение в набор данных **`failed`**.  

##### b. Напечатайте набор данных **`passed`**, чтобы проверить наличие в нём 12 наблюдений. 

In [6]:
data work.passed work.failed;
    set orion.test_answers;
    title "Passed result";
    array _true_answers {10}  $ _temporary_ ('A', 'C', 'C', 'B', 'E', 'E', 'D', 'B', 'B', 'A');
    array _answers {10} $ Q1-Q10;
    res = 0;
    do i = 1 to 10;
        if _true_answers{i} = _answers{i} then
            res = res + 1;
    end;
    if res >= 7 then output work.passed;
    else output work.failed;
    drop i res;
run;    

proc print data=passed;
run;

Obs,Employee_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
1,121044,A,C,C,B,D,E,D,B,B,A
2,120145,B,C,C,,E,E,D,B,A,A
3,121107,A,C,C,B,E,E,D,B,B,A
4,121038,B,C,C,B,D,D,D,B,B,A
5,120273,C,C,C,B,E,E,E,B,B,A
6,120759,A,C,C,B,E,E,D,B,B,A
7,121030,C,C,C,C,E,E,D,B,B,B
8,121017,B,B,E,B,E,E,D,B,B,A
9,121062,A,C,C,B,E,E,D,B,B,A
10,121119,C,C,C,B,E,E,D,B,B,A
